In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset , DataLoader
import torch.nn as nn 
import torch.optim as optim
import matplotlib.pyplot as plt 

In [3]:
# set random seed for reproducibility 
torch.manual_seed(42)

In [4]:
df = pd.read_csv("fmnist_small.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,...,pixel717,pixel718,pixel719,pixel720,pixel721,pixel722,pixel723,pixel724,pixel725,pixel726,pixel727,pixel728,pixel729,pixel730,pixel731,pixel732,pixel733,pixel734,pixel735,pixel736,pixel737,pixel738,pixel739,pixel740,pixel741,pixel742,pixel743,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,72,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,120,108,0,0,4,6,11,17,80,24,224,30,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,4,1,0,0,200,226,224,237,0,0,0,0,0,30,109,199,243,255,255,251,242,236,230,246,228,0,0,3,0,2,0,0,210,228,228,233,0,0,0,0,0,0,0,0,0,31,81,133,184,201,190,117,0,0,2,1,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,43,117,34,15,24,33,117,80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,51,144,152,202,213,210,205,204,221,157,172,131,55,0,0,0,1,0,0,0,0,0,0,1,0,1,96,156,156,152,140,...,156,153,156,149,149,47,0,6,0,0,0,0,0,0,0,0,0,5,0,77,157,148,155,146,151,149,152,154,157,158,161,148,159,58,0,6,0,0,0,0,0,0,0,0,0,4,0,60,143,143,148,146,152,152,148,148,147,145,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,0,0,0,2,0,33,114,37,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,47,136,31,136,42,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,212,215,213,209,207,204,204,204,204,199,197,94,0,3,0,9,164,152,138,152,160,152,162,144,208,181,1,18,11,17,13,13,11,10,7,5,5,5,7,0,0,0,1,0,0,41,69,88,86,94,106,114,118,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,0,2,0,58,145,114,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,29,132,0,0,101,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,132,134,151,170,188,209,230,136,0,0,0,0,0,0,0,0,18,134,155,178,149,163,165,138,147,170,149,134,165,153,155,134,143,172,215,62,0,0,0,0,0,0,0,0,10,190,178,194,209,211,209,205,211,215,213,217,225,228,213,203,174,151,188,10,0,0,0,0


In [5]:
df.shape

(6000, 785)

In [6]:
# seperate target andf input features
X = df.drop(columns = ['label'] , axis = 1)
y = df['label']

In [7]:
# do the train test split 
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 42)

In [8]:
# normalize the pixcel values
X_train = X_train / 255.0
X_test = X_test / 255.0

In [11]:
# convert all these into numpy array 
X_train = X_train.to_numpy()
X_test = X_test.to_numpy() 
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [14]:
class CustomDataset(Dataset): 
    def __init__(self , features , labels):
        # convert the input numpy array into tensor and also from 1D to 2D
        # reshape(batch_size = -1[place holder] , number of channel , Height , Width)
        self.features = torch.tensor(features , dtype = torch.float32).reshape(-1 , 1 , 28 , 28)
        self.labels = torch.tensor(labels , dtype = torch.long)
    
    def __len__(self): 
        return len(self.features) 
    
    def __getitem__(self , index): 
        return self.features[index] , self.labels[index] 

In [15]:
train_dataset = CustomDataset(X_train , y_train)
test_dataset = CustomDataset(X_test , y_test)

In [16]:
# make the dataloader 
train_data_loader = DataLoader(
    dataset = train_dataset, 
    batch_size = 32, 
    shuffle = True,
    pin_memory = True
)

test_data_loader = DataLoader(
    dataset = test_dataset, 
    batch_size = 32, 
    pin_memory = True
)

In [17]:
# Define the CNN 
class MyCNN(nn.Module): 
    def __init__(self , input_channels): 
        super().__init__()
        # CNN has 2 parts: 1. Feature Extraction Layers , 2. FC layers
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels = input_channels , out_channels = 32 , kernel_size = 3 , padding = 'same'),
            nn.BatchNorm2d(num_features = 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2 , stride = 2),

            nn.Conv2d(in_channels = 32 , out_channels = 64 , kernel_size = 3 , padding = 'same'),
            nn.BatchNorm2d(num_features = 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2 , stride = 2)
        )
        self.classifier = nn.Sequential(
            # flatten layer 
            nn.Flatten(),
            nn.Linear(in_features = 64*7*7 , out_features = 128),
            nn.ReLU(), 
            nn.Dropout(p = 0.4),

            nn.Linear(in_features = 128 , out_features = 64),
            nn.ReLU(),
            nn.Dropout(p = 0.4),

            nn.Linear(64 , 10)
        )

    def forward(self , x): 
        x = self.feature_extraction(x)
        x = self.classifier(x)
        return x 

In [18]:
learning_rate = 0.01
epochs = 100

In [19]:
model = MyCNN(input_channels = 1)
model.to(device)

MyCNN(
  (feature_extraction): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters() , lr = learning_rate , weight_decay = 1e-4)

In [21]:
for epoch in range(epochs): 
    total_epoch_loss = 0
    for batch_features , batch_labels in train_data_loader: 
        # move the features and labels to GPU 
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        # forward pass 
        outputs = model(batch_features)
        # calculate loss 
        loss = criterion(outputs , batch_labels)
        total_epoch_loss += loss.item()

        # clear grad 
        optimizer.zero_grad()
        # backward 
        loss.backward()
        # update weights and bias 
        optimizer.step()

    avg_epoch_loss = total_epoch_loss / len(train_data_loader)
    print(f"Epoch: {epoch + 1} | Loss: {avg_epoch_loss}")

Epoch: 1 | Loss: 1.6670839246114095
Epoch: 2 | Loss: 0.9905066915353139
Epoch: 3 | Loss: 0.8119085093339284
Epoch: 4 | Loss: 0.7214896335204443
Epoch: 5 | Loss: 0.6658143840233485
Epoch: 6 | Loss: 0.6162446649869283
Epoch: 7 | Loss: 0.572268722653389
Epoch: 8 | Loss: 0.5459293020764987
Epoch: 9 | Loss: 0.5194110749165217
Epoch: 10 | Loss: 0.48124695946772894
Epoch: 11 | Loss: 0.4710633987188339
Epoch: 12 | Loss: 0.4547058030962944
Epoch: 13 | Loss: 0.42825803369283677
Epoch: 14 | Loss: 0.403708356320858
Epoch: 15 | Loss: 0.3856340597569943
Epoch: 16 | Loss: 0.36812804996967313
Epoch: 17 | Loss: 0.35010647306839626
Epoch: 18 | Loss: 0.32722114597757657
Epoch: 19 | Loss: 0.3223238901297251
Epoch: 20 | Loss: 0.3017922617991765
Epoch: 21 | Loss: 0.284533609499534
Epoch: 22 | Loss: 0.26595493465662
Epoch: 23 | Loss: 0.27313355877995493
Epoch: 24 | Loss: 0.2538362294435501
Epoch: 25 | Loss: 0.240575590133667
Epoch: 26 | Loss: 0.23100154941280682
Epoch: 27 | Loss: 0.21541160581012567
Epoch: 2

In [22]:
model.eval()

MyCNN(
  (feature_extraction): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [24]:
total = 0
correct = 0

with torch.no_grad(): 
    for batch_features , batch_labels in test_data_loader:
        # move the features and labels to GPU 
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        # forward pass 
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs , 1)
        total = total + batch_labels.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()

print(f"Accuracy: {correct/total}")

Accuracy: 0.8733333333333333


In [26]:
total = 0
correct = 0

with torch.no_grad(): 
    for batch_features , batch_labels in train_data_loader:
        # move the features and labels to GPU 
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        # forward pass 
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs , 1)
        total = total + batch_labels.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()

print(f"Accuracy: {correct/total}")

Accuracy: 1.0
